In [1]:
from pathlib import Path
from tkinter import Tk
from tkinter.filedialog import askopenfilenames
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.ticker as plticker

idx = pd.IndexSlice

# Define automatic naming format from file_paths
def get_name(file_path):
    name = file_path.stem.split("_")
    #name = name[4]
    name = name[3]+'_'+name[4]
    return name

# Define axis labels based on parameter
def gen_param_label(param):
    param_dict = {
        'mAh':'[mAh]',
        'mass':'[mAh/g]',
        'areal':r'$\mathrm{[mAh/cm^2]}$',
        'volume':r'$\mathrm{[mAh/cm^3]$'}
    
    return param_dict[param]

In [ ]:
# Open file dialog
root = Tk()
root.attributes("-topmost", True)
root.withdraw()  # stops root window from appearing
file_paths = askopenfilenames(title="Select files",
                              filetypes=[("Comma-separated values", "*.csv")])  # "Open" dialog box and return the selected path

# Convert filepaths to pathlib format
filez = []
for file in file_paths:
    file = Path(file)
    filez.append(file)
    
file_paths = filez

In [ ]:
# Generate 'names' and 'param' dictionaries from file_paths
# read_csv to dataframe
names={}
params = {}
df=[]
for i, file_path in enumerate(file_paths):
    name = get_name(file_paths[i])
    names[i] = name
    
    df_imp = pd.read_csv(file_paths[i],header=[1,2],index_col=0)
    df_imp = pd.concat([df_imp], axis=1, keys=[i])
    
    df.append(df_imp)

    params[i] = df_imp.columns.get_level_values(1)[0]
      
df = pd.concat(df, axis=1)
df.sort_index(inplace=True)

for cell in names:
    print(cell, '->', names[cell], '->', params[cell])

In [ ]:
# Make list of cycles to plot
#cycles = df.index.unique(level=1).tolist()
cycles = range(1,5)

# Create figure and axes
plt.rcParams.update({'font.size': 14})
fig, ax = plt.subplots(figsize=(5.5, 5.5))

# Params
ax.set_ylim(ymin=290, ymax=350)
ax.set_xlim(xmin=0, xmax=3)

# Plot cycle data
for cell in names:
    df_plot = df[cell][params[cell]]['p_cap']
    
    if 'p_cap_std' in df[cell][params[cell]].columns:
        yerr = df[cell][params[cell]]['p_cap_std']
        plt.errorbar(df_plot.index,
                     df_plot,
                     yerr=yerr,
                     fmt='o',
                     capsize=4, 
                     elinewidth=2,
                     markeredgewidth=2,
                     label = names[cell])    
    else:
        plt.scatter(df_plot.index,df_plot,label = names[cell])
        
loc = plticker.MultipleLocator(base=1.0) # this locator puts ticks at regular intervals
ax.xaxis.set_major_locator(loc)
    
plt.xlabel("Cycle")
plt.ylabel('Capacity '+gen_param_label(params[0]))
plt.legend(loc='upper left')

plt.savefig(file_paths[cell].parent.joinpath('{}_{}.svg'.format('cycles_mass',names[cell])),
            format='svg', 
            dpi=600, 
            bbox_inches = 'tight')
plt.show()